In [1]:
import os
os.environ['TRKXINPUTDIR']="/global/cfs/projectdirs/m3443/usr/caditi97/iml2020/misaligned/new_mis/shift_x_v12/1"
os.environ['TRKXOUTPUTDIR']= "/global/cfs/projectdirs/m3443/usr/caditi97/iml2020/misaligned/new_mis/shift_x_v12_pre/1"

In [2]:
import pkg_resources
import yaml
import pprint
import random
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import tqdm
from os import listdir
from os.path import isfile, join
import matplotlib.cm as cm
import sys
# %matplotlib widget

sys.path.append('/global/homes/c/caditi97/exatrkx-iml2020/exatrkx/src/')

# 3rd party
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from trackml.dataset import load_event
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint


# local import
from exatrkx import config_dict # for accessing predefined configuration files
from exatrkx import outdir_dict # for accessing predefined output directories
from exatrkx.src import utils_dir
from exatrkx.src import utils_robust
from utils_robust import *


# for preprocessing
from exatrkx import FeatureStore
from exatrkx.src import utils_torch

# for embedding
from exatrkx import LayerlessEmbedding
from exatrkx.src import utils_torch
from torch_cluster import radius_graph
from utils_torch import build_edges
from embedding.embedding_base import *

# for filtering
from exatrkx import VanillaFilter

# for GNN
import tensorflow as tf
from graph_nets import utils_tf
from exatrkx import SegmentClassifier
import sonnet as snt

# for labeling
from exatrkx.scripts.tracks_from_gnn import prepare as prepare_labeling
from exatrkx.scripts.tracks_from_gnn import clustering as dbscan_clustering

# track efficiency
from trackml.score import _analyze_tracks
from exatrkx.scripts.eval_reco_trkx import make_cmp_plot, pt_configs, eta_configs
from functools import partial

In [3]:
action = 'build'

config_file = pkg_resources.resource_filename(
                    "exatrkx",
                    os.path.join('configs', config_dict[action]))
with open(config_file) as f:
    b_config = yaml.load(f, Loader=yaml.FullLoader)
    
pp = pprint.PrettyPrinter(indent=4)

In [4]:
b_config['pt_min'] = 0
b_config['endcaps'] = True
b_config['n_workers'] = 2
b_config['n_files'] = 10
b_config['noise'] = 0
pp.pprint(b_config)

{   'adjacent': True,
    'cell_information': True,
    'endcaps': True,
    'layerless': True,
    'layerwise': False,
    'n_files': 10,
    'n_tasks': 1,
    'n_workers': 2,
    'noise': 0,
    'pt_min': 0}


In [5]:
%%time
preprocess_dm = FeatureStore(b_config)
preprocess_dm.prepare_data()

Loading detector...
Failed to load preprocessed detector. Building...
  Extracting thicknesses...
  Done.
  Extracting rotations...
  Done.
Index(['volume_id', 'layer_id', 'module_id', 'cx', 'cy', 'cz', 'rot_xu',
       'rot_xv', 'rot_xw', 'rot_yu', 'rot_yv', 'rot_yw', 'rot_zu', 'rot_zv',
       'rot_zw', 'module_t', 'module_minhu', 'module_maxhu', 'module_hv',
       'pitch_u', 'pitch_v'],
      dtype='object')
  Extracting thicknesses...
  Done.
Detector preprocessed and saved.
Writing outputs to /global/cfs/projectdirs/m3443/usr/caditi97/iml2020/misaligned/new_mis/shift_x_v12_pre/1/feature_store
Invalid input: /global/cfs/projectdirs/m3443/usr/caditi97/iml2020/misaligned/new_mis/shift_x_v12/1/detectors.csv
Preparing 1000
Preparing 1001
Layerless truth graph built for /global/cfs/projectdirs/m3443/usr/caditi97/iml2020/misaligned/new_mis/shift_x_v12/1/event000001001 with size (2, 91386)
Cell features for 1001
Loading event /global/cfs/projectdirs/m3443/usr/caditi97/iml2020/misaligned/